In [1]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error,r2_score
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
import os
import random
import warnings
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression


from sklearn.utils import shuffle
warnings.filterwarnings("ignore")
def seed_everything(seed=555):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # torch.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
seed_everything()

In [3]:
def r2(y,yhat):
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat-ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    return 1-ssreg / sstot

In [4]:
from mlxtend.evaluate import PredefinedHoldoutSplit
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import joblib
rf = RandomForestRegressor()

In [5]:
filenames=[ '4.csv.csv',
 '5.csv.csv',
 '6.csv.csv',
 '8.csv.csv',
 '9.csv.csv',
 'C.csv.csv',
 'F.csv.csv',
 'P.csv.csv',
 'X.csv.csv']

In [4]:
for file in filenames:
    df=pd.read_csv("../input/500feature-selection/"+file)
    y=df.iloc[:,0]
    x=df.iloc[:,1:64]
    x=(x-x.min())/(x.max()-x.min())
    rng = np.random.RandomState(123)
    my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
    piter = PredefinedHoldoutSplit(my_validation_indices)
    sfs1 = SFS(rf, 
           forward=True, 
           k_features=(1,x.shape[1]), 
           floating=False, 
           verbose=2,
           scoring="r2",
           cv=piter)

    sfs1 = sfs1.fit(x, y)
    pd.DataFrame.from_dict(sfs1.get_metric_dict()).T.to_csv(file+".csv")

    #lr是一个LogisticRegression模型
    joblib.dump(sfs1, file+'.model')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   10.7s finished

[2021-08-06 03:57:11] Features: 1/63 -- score: 0.7994211100942552[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  62 out of  62 | elapsed:   10.6s finished

[2021-08-06 03:57:21] Features: 2/63 -- score: 0.8276630112352996[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  61 out of  61 | elapsed:   10.4s finished

[2021-08-06 03:57:32] Features: 3/63 -- score: 0.8348061621518794[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [6]:
clf4 = joblib.load("4.csv.model")
clf5 = joblib.load("5.csv.model")
clf6 = joblib.load("6.csv.model")
clf8 = joblib.load("8.csv.model")
clf9 = joblib.load("9.csv.model")
clfc = joblib.load("C.csv.model")
clff = joblib.load("F.csv.model")
clfp = joblib.load("P.csv.model")
clfx = joblib.load("X.csv.model")

## 4.csv

In [48]:
df=pd.read_csv('./数据/4.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"4.csv")
x=x.loc[:,clf4.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]

In [9]:
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]

### LGBMRegressor

In [10]:
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
r21

0.8856882838441695

### SVR

In [23]:
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
r22

0.942287056598479

In [25]:
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"4.csv")

## 5.csv

In [49]:
df=pd.read_csv('./数据/5.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"5.csv")
x=x.loc[:,clf5.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"5.csv")

0.9975127372318804

0.8743529533833014

## 6.csv

In [50]:
df=pd.read_csv('./数据/6.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"6.csv")
x=x.loc[:,clf6.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"6.csv")

0.9738856981662435

0.8504168627338993

## 8.csv

In [51]:
df=pd.read_csv('./数据/8.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"8.csv")
x=x.loc[:,clf8.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"8.csv")

0.9074350764526021

0.7708593695930395

## 9.csv

In [52]:
df=pd.read_csv('./数据/9.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"9.csv")
x=x.loc[:,clf9.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"9.csv")

0.9909191272214729

0.9549799012946049

## C.csv

In [53]:
df=pd.read_csv('./数据/C.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"C.csv")
x=x.loc[:,clfc.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"C.csv")

0.6521735235491934

0.8442405924003471

## F.csv

In [54]:
df=pd.read_csv('./数据/F.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"F.csv")
x=x.loc[:,clff.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"F.csv")

0.9142389705866958

0.8798561687055776

## P.csv

In [55]:
df=pd.read_csv('./数据/P.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"P.csv")
x=x.loc[:,clfp.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"P.csv")

0.8571051591570584

0.9321630445245049

## X.csv

In [56]:
df=pd.read_csv('./数据/X.csv')
y=df.iloc[:,0]
x=df.iloc[:,1:64]
rf.fit(x,y)
fea=pd.DataFrame()
fea["name"]=x.columns
fea["imp"]=rf.feature_importances_
fea.to_csv("imp_"+"X.csv")
x=x.loc[:,clfx.k_feature_names_]
x=(x-x.min())/(x.max()-x.min())
rng = np.random.RandomState(123)
my_validation_indices = rng.permutation(np.arange(df.shape[0]))[:int(df.shape[0]*0.3)]
test_index=list(my_validation_indices)
train_index=list(x.index)
for i in test_index:
    train_index.remove(i)
xtrain,ytrain,xtest,ytest=x.iloc[train_index,:],y[train_index],x.iloc[test_index,:],y[test_index]
model1 = LGBMRegressor()
model1.fit(xtrain,ytrain)
ypred1=model1.predict(xtest)
r21=r2(ytest,ypred1)
display(r21)
model2=SVR()
model2.fit(xtrain,ytrain)
ypred2=model2.predict(xtest)
r22=r2(ytest,ypred2)
display(r22)
output=pd.DataFrame()
output["ytest"]=ytest
output["y_pred_lgb"]=ypred1
output["y_pred_svr"]=ypred2
output.to_csv("output_"+"P.csv")

0.9467239201599377

0.897475017790724